In [1]:
root='/kaggle/working/'
import sys
!git clone https://github.com/nudojacopo/thesis.git
module_path = root+'thesis/src/SYN/SYN_package'
sys.path.append(module_path)
sys.path.append(module_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
!pip install powerlaw
from functions import *
from estimate_parameters import *
import json
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import powerlaw
import warnings
!pip install gdown
import gdown

url='https://drive.google.com/uc?id=1LUaN2XL3o9TIggKTxcNkmIGSdBx01Hqo'
output='facebook.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=1Dq5Z8KjEhbiUNTd8vgxX0ZK6aP_a7h4I'
output='voat.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=1jXcIGv2zoTZeuj8W2EPXyhQLOaiRU2Tn'
output='gab.csv'
gdown.download(url,output,quiet=False)

url='https://drive.google.com/uc?id=1qCPKDWjl9CxaRf8EJK5oFGvwae0xC9-q'
output='reddit.csv'
gdown.download(url,output,quiet=False)

Cloning into 'thesis'...
remote: Enumerating objects: 1573, done.
remote: Counting objects: 100% (300/300), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 1573 (delta 186), reused 260 (delta 167), pack-reused 1273
Receiving objects: 100% (1573/1573), 72.40 MiB | 31.86 MiB/s, done.
Resolving deltas: 100% (1007/1007), done.


In [ ]:
fb=pd.read_csv('/kaggle/working/facebook.csv')
rd=pd.read_csv('/kaggle/working/reddit.csv')
vo=pd.read_csv('/kaggle/working/voat.csv')
gb=pd.read_csv('/kaggle/working/gab.csv')

h=100
n=1000000

In [ ]:
fb=filter_first_h(fb,h)

gb=filter_first_h(gb,h)

rd=filter_first_h(rd,h)

vo=filter_first_h(vo,h)

names = ['gb','rd','fb','vo']
datas = [gb,rd,fb,vo]

params_dict = process_social_platform(names, datas)

In [ ]:
time_intervals = np.arange(0, 1, 0.05)


all_dfs = []
for i, df in enumerate(datas):
    social = names[i]
    parameters = params_dict[social]

    simulated_t, observed_t = simulate_data(df, parameters, activate_tqdm=True, num_threads=n)# peensaree a come limitar i valori dell funzionee IAT senza crear break infiniti
    globals()[f"{social}s_M1"] = simulated_t
    globals()[f"{social}o_M1"] = observed_t
    df_simulated = pd.DataFrame(simulated_t, columns=[f"{social}s_M1_col{i}" for i in range(simulated_t.shape[1])])
    df_observed = pd.DataFrame(observed_t, columns=[f"{social}o_M1_col{i}" for i in range(observed_t.shape[1])])

    df_simulated.to_csv(f"{social}s_M1.csv", index=False)
    df_observed.to_csv(f"{social}o_M1.csv", index=False)

    simulated_t_ECDF = calculate_ECDF(simulated_t, time_intervals)
    simulated_t_ECDF['Platform'] = social
    simulated_t_ECDF['Style'] = 'Simulated'

    observed_t_ECDF = calculate_ECDF(observed_t, time_intervals)
    observed_t_ECDF['Platform'] = social
    observed_t_ECDF['Style'] = 'Observed'

    df_platform = pd.concat([simulated_t_ECDF, observed_t_ECDF], ignore_index=True)
    all_dfs.append(df_platform)

df_all_M1 = pd.concat(all_dfs, ignore_index=True)


Dynamics

In [ ]:
# Prepare data
palette = {'rd': 'purple', 'vo': 'green', 'fb': 'blue', 'yt': 'red','gb':'brown'}
df_all = df_all_M1.sort_values(by='Style')

# Plot comment arrival dynamic
plt.figure(figsize=(12, 8))
sns.lineplot(data=df_all_M1, x='Time Grid Value', y='Share', hue='Platform', style='Style', err_style='band', errorbar=('ci', 80), palette=palette)
plt.title('Distribution of Conversation Lifetime on Social Platforms')
plt.ylabel('Lifetime (minutes)')
plt.xlabel('Percentile')
plt.grid(False)
plt.legend(title='Platform')
plt.show()

# Calculate Loss
df_line1 = df_all_M1[df_all_M1['Style'] == 'Simulated']
df_line2 = df_all_M1[df_all_M1['Style'] == 'Observed']
df_line1 = df_line1.set_index(['Time Grid Value', 'Platform'])
df_line2 = df_line2.set_index(['Time Grid Value', 'Platform'])
df_merged = df_line1.join(df_line2, lsuffix='_line1', rsuffix='_line2')
mse_per_platform_M1 = df_merged.groupby('Platform').apply(lambda x: np.mean((x['Share_line1'] - x['Share_line2'])**2))

Participation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # tqdm for progress bar

# Assuming rd, fb, vo, yt are your DataFrames

# Add 'platform' column to each DataFrame
rdo_M1['platform'] = 'Reddit'
voo_M1['platform'] = 'Voat'
fbo_M1['platform'] = 'Facebook'
gbo_M1['platform'] = 'Gab'
# Combine all DataFrames into a single DataFrame
all_data = []

# Process each dataset individually
for df in tqdm([rdo_M1,  voo_M1,fbo_M1,gbo_M1]):#yto
    df['number_of_comments'] = df.groupby('post_id')['post_id'].transform('count')
    df['sequential_position'] = df.groupby('post_id')['temporal_distance_birth_base_100h'].rank(method='first')

    # Determine the number of bins
    num_bins = 30

    # Calculate the bin of position
    df['bin_of_position'] = ((df['sequential_position'] - 1) / df['number_of_comments'] * num_bins).astype(int) + 1
    df['first_comment']=df['sequential_number_of_comment_by_user_in_thread']==1
    # Group by 'bin_of_position', 'post_id', and calculate unique users per post
    df['first_comment'] = df.groupby(['bin_of_position', 'post_id'])['first_comment'].transform('mean')
    all_data.append(df)
all_data = pd.concat(all_data, ignore_index=True)
# Aggregate data across all datasets
agg_activity_obs = all_data.groupby(['platform', 'bin_of_position', 'post_id'])['first_comment'].mean().reset_index()
plt.figure(figsize=(12, 8))
sns.lineplot(
    data=agg_activity_obs,
    x='bin_of_position',
    y='first_comment',
    hue='platform',  # Different line for each platform
    estimator='mean',  # or 'sum', 'median', etc. depending on your aggregation preference
    errorbar=('ci', 80),  # 99.999% confidence interval
    linewidth=3,
    palette={'Reddit': 'purple', 'Voat': 'green', 'Facebook': 'blue','Gab': 'brown'}#,'YouTube':'red'}  # Custom palette with specified colors
)
plt.title('Observed')
plt.ylabel('Share of first commentt')
plt.xlabel('Bin of Position')
plt.grid(False)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # tqdm for progress bar

# Assuming rd, fb, vo, yt are your DataFrames

# Add 'platform' column to each DataFrame
rds_M1['platform'] = 'Reddit'
vos_M1['platform'] = 'Voat'
fbs_M1['platform'] = 'Facebook'
gbs_M1['platform'] = 'Gab'
# Combine all DataFrames into a single DataFrame
all_data = []

# Process each dataset individually
for df in tqdm([rds_M1,  vos_M1,fbs_M1,gbs_M1]):#yto
    df['number_of_comments'] = df.groupby('post_id')['post_id'].transform('count')
    df['sequential_position'] = df.groupby('post_id')['temporal_distance_birth_base_100h'].rank(method='first')

    # Determine the number of bins
    num_bins = 30

    # Calculate the bin of position
    df['bin_of_position'] = ((df['sequential_position'] - 1) / df['number_of_comments'] * num_bins).astype(int) + 1

    df['first_comment']=df['sequential_number_of_comment_by_user_in_thread']==1
    # Group by 'bin_of_position', 'post_id', and calculate unique users per post
    df['first_comment'] = df.groupby(['bin_of_position', 'post_id'])['first_comment'].transform('mean')
    all_data.append(df)
all_data = pd.concat(all_data, ignore_index=True)
# Aggregate data across all datasets
agg_activity_sim_M1 = all_data.groupby(['platform', 'bin_of_position', 'post_id'])['first_comment'].mean().reset_index()

plt.figure(figsize=(12, 8))

# Create the line plot
sns.lineplot(
    data=agg_activity_sim_M1,
    x='bin_of_position',
    y='first_comment',
    hue='platform',  # Different line for each platform
    estimator='mean',  # Aggregation preference
    errorbar=('ci', 80),  # 80% confidence interval
    linewidth=3,
    palette={'Reddit': 'purple', 'Voat': 'green', 'Facebook': 'blue', 'Gab': 'brown'},  # Adding style for dashed line
)

# Manually setting the line style for one of the platforms
lines = plt.gca().get_lines()
for line in lines:
  line.set_linestyle('--')

plt.title('Simulated - M1')
plt.ylabel('Average Unique Users per Post')
plt.xlabel('Bin of Position')
plt.grid(False)
plt.show()

